## Day 2 Task: Creating a map from data.

This task is to teach you how to bring data from a satellite file and locate it on a map of the Earth. Much of the heavy lifting here is done within the basemap or cartopy libraries (see notebooks/cartopyGuide.ipynb for examples). But working with geospatial data invariably means you need to show where the data exists on a map.

In [ ]:
# Libraries used

import numpy as np
import netCDF4 as nc4
from netCDF4 import Dataset

### Reading a NetCDF file

This will be similar to what we did earlier with HDF files. Our example file today comes from the TROPOMI instrument.

In [ ]:

my_example_nc_file = 'S5P_OFFL_L2__NO2____20220116T180124_20220116T194254_22081_02_020301_20220118T102523.nc'
fh = Dataset(my_example_nc_file, mode='r')
print (fh)


### Getting groups and subgroups in NetCDF

The print statement above will have produced a long set of attributes containing information about the file (when it was created, what version of the algorithm was used, ...). But we want to get some data! Let's look at the top-level groups and one level beneath that. The output is slightly different from what we saw with the HDF example, but it is close enough that you should get the general idea.

In [ ]:
print (fh.groups)

print (fh.groups['PRODUCT'])


### Finding the data values needed to make a map

If we want a simple map, we probably want to know where our data is located (its latitude and longitude), and then some data values to put into the map (we will use "nitrogendioxide_tropospheric_column" as an example). First we will list out the names of the variables contained in our "PRODUCT" group, then select the ones we would like and read them into our code.

In [ ]:
print (fh.groups['PRODUCT'].variables.keys())

#By printing the .variables.keys() we can see that this .nc file has the following variables:

In [ ]:
# If we select one of these variables, say the ‘nitrogendioxide_tropospheric_column’ using this command

print (fh.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'])

# we can see attributes about this variable

In [ ]:
# NetCDF variables read in directly as numpy arrays, which we can slice as usual
# We select the 0 index in the first dimension because we only want the first "time" element
# (there is only one of these anyhow!)

lons = fh.groups['PRODUCT'].variables['longitude'][:][0,3000:3500,:]
lats = fh.groups['PRODUCT'].variables['latitude'][:][0,3000:3500,:]
no2 = fh.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'][0,3000:3500,:]

# All our arrays should have the same shape at this point

print (lons.shape)
print (lats.shape)
print (no2.shape)

In [ ]:
no2_units = fh.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'].units
# this line gets the units of NO2 so we can add it to the plot later on
print (no2_units)

### Let's make a map!

Now we can create an instance of our map and start to plot things on it. The mapping library (cartopy) has many features that can be added here, including colorbars, gridlines, geographical features like country borders, and more.

In [ ]:
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt

fig = plt.figure()

ax = plt.axes(projection=ccrs.PlateCarree())

cmap = mpl.cm.Oranges
norm = mpl.colors.Normalize(vmin=0, vmax=0.001)

# note that we transform our data with the same projection as we are plotting
mesh = plt.pcolormesh(lons, lats, no2, shading='nearest', cmap=cmap, norm=norm, transform=ccrs.PlateCarree())

# Add a colorbar for the mesh
fig.colorbar(mpl.cm.ScalarMappable(cmap=cmap, norm=norm), orientation='horizontal')

ax.coastlines()

ax.set_extent([-120, -70, 20, 50])

plt.show()


## Try it yourself!

#### 1. Open the example NetCDF file, or download your own from search.earthdata.nasa.gov

    The example file is similar to the dataset called "Sentinel-5P TROPOMI Tropospheric NO2 1-Orbit L2 5.5km x 3.5km V2 (S5P_L2__NO2____HiR) at GES DISC"

#### 2. Read the variables in your file and list them

    The TROPOMI files contain many more datasets than the VIIRS files we saw earlier. This is partly because sensing the amount of gas in the atmosphere is a more complicated process than measuring the radiance at the surface. Explore different groups within the file. Can you find the dataset for water vapour slant column density?

#### 3. Make a global map of "water_slant_column_density"

    This is an intermediate step in getting the NO2 amount, but it is still an interesting thing to map. In the example above, we used set_extent() to clip the map axis to a specific latitude/longitude region. We also sliced the data, taking only scanlines 3000 to 3500. Try reading in all of the water column data and all of the latitude/longitude data, and plotting in on a global map. Does the satellite actually see more over the poles? Can you find a map projection that proves this is not the case?
    
#### 4. Make a regional map of nitrogen dioxide

    Let's go back to the tropospheric vertical column of nitrogen dioxide ("nitrogendioxide_tropospheric_column"). What does this data look like over Alberta? Zoom your map way in to the point where you can see individual pixels (each is about 5.5 by 3.5 km). Try overlaying some map features (e.g. provincial borders, roads, rivers) to give your data some regional context.

    What happens to the map over Alberta if you screen out data with a quality flag ("qa_value") less than 0.5?
    
    What if you plot the ratio of the signal ("nitrogendioxide_tropospheric_column") to its precision ("nitrogendioxide_tropospheric_column_precision")?
    
    Can you make a map where you use the latitude/longitude boundaries of each ground pixel instead of the latitude/longitude of the pixel centre?

